### Génération d'un tableau de coordonnées fictives

In [12]:
import numpy as np

molecules_nb = 10

# Génération d'un vecteur de positions. Pour chaque molécule, le vecteur contient 3 coordonnées x, y, z,
# chacune dans l'intervalle [-1,1[
positions = np.random.random_sample(molecules_nb*3) * 2 - 1
positions

array([ 0.05630524, -0.22366857, -0.60286125, -0.99615371, -0.52112679,
       -0.29220214, -0.27122452, -0.14906277,  0.6915369 , -0.51938744,
       -0.835469  , -0.05590557, -0.42933313, -0.52677007, -0.90592153,
        0.85469826,  0.63221378,  0.46476837, -0.67978513,  0.21617408,
       -0.51060904,  0.17133427,  0.07713589, -0.88660206,  0.0831478 ,
       -0.18664951,  0.38568355, -0.06703106, -0.37996184,  0.46636157])

### Conversion des coordonnées en distances

#### Ajout d'une molécule fictive

In [13]:
def ajout_mol_fictive(positions):
    """ Ajoute une molécule fictive aux positions initiales. Pour le moment, la place simplement
    aux coordonnées (0,0,0). Dans le futur, elle la placera à une position n'appartenant pas avec
    certitude au plan formé par les trois premières molécules"""
    return np.append(positions, [0,0,0])
    
positions = ajout_mol_fictive(positions)
positions

array([ 0.05630524, -0.22366857, -0.60286125, -0.99615371, -0.52112679,
       -0.29220214, -0.27122452, -0.14906277,  0.6915369 , -0.51938744,
       -0.835469  , -0.05590557, -0.42933313, -0.52677007, -0.90592153,
        0.85469826,  0.63221378,  0.46476837, -0.67978513,  0.21617408,
       -0.51060904,  0.17133427,  0.07713589, -0.88660206,  0.0831478 ,
       -0.18664951,  0.38568355, -0.06703106, -0.37996184,  0.46636157,
        0.        ,  0.        ,  0.        ])

#### 